In [ ]:
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 34.8 MB/s eta 0:00:00


In [ ]:
#IMPORTS PARA TRABALHAR COM DADOS GEORREFERENCIADOS
import os
import rasterio as rio
from rasterio.plot import *
import numpy as np
import torch
import matplotlib.pyplot as plt;

In [ ]:
#IMPORTS PARA TRABALHAR COM REDES NEURAIS
from torch import nn, optim
from torch.utils.data import TensorDataset, Dataset, DataLoader
from torch.utils.data import SubsetRandomSampler
from torchvision import transforms
from random import sample

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# Set the directory path where the geospatial images are stored
diretorio_barbacena = "/content/gdrive/MyDrive/dados_redes_neurais_imagens/imgs_BQ_NN";
diretorio_rioverde = "/content/gdrive/MyDrive/dados_redes_neurais_imagens/imgs_RioVerde_NN";
imagens_barbacena = [];
imagens_rioverde = [];

# Loop through all the files in the directory
for filename in os.listdir(diretorio_barbacena):

    if filename.endswith(".tif") or filename.endswith(".tiff"):
      
        with rio.open(os.path.join(diretorio_barbacena, filename)) as src:
            imagens_barbacena.append(src.read())
            #print(type(src))           #TIPO DATASET RASTERIO
            #print(type(src.read()))    #TIPO NP.NDARRAY
            #show(src.read())
          
for filename in os.listdir(diretorio_rioverde):

    if filename.endswith(".tif") or filename.endswith(".tiff"):
      
        with rio.open(os.path.join(diretorio_rioverde, filename)) as src:
            imagens_rioverde.append(src.read())


In [ ]:
#TRANSFORMANDO OS NAN EM ZEROS
for i in range(len(imagens_barbacena)):
  imagens_barbacena[i] = np.nan_to_num(imagens_barbacena[i], nan = 0)
  #print(imagens_treino[i])


In [ ]:
print(type(imagens_barbacena[0]), type(imagens_barbacena))


<class 'numpy.ndarray'> <class 'list'>


In [ ]:
#NÃO TENHO GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
#NORMALIZANDO O TAMANHO DAS IMAGENS
fixed_size = (3, 400, 600)
imagens_barbacena = [np.transpose(np.resize(np.transpose(img, (1, 2, 0)), fixed_size), (0, 1, 2)) for img in imagens_barbacena]
#imagens_barbacena = []; 
print(imagens_barbacena[1].shape)
imagens_rioverde = [np.transpose(np.resize(np.transpose(img, (1, 2, 0)), fixed_size), (0, 1, 2)) for img in imagens_rioverde]

#CONVERTETNDO A LISTA PARA TENSORES
#train_tensor = torch.stack([torch.Tensor(i) for i in imagens_barbacena_resized])
#test_tensor = torch.stack([torch.Tensor(i) for i in imagens_rioverde_resized])

(3, 400, 600)


In [ ]:
#DEFININDO OS DATASETS DE TREINO E TESTE E DEPOIS USANDO-OS PARA CARREGAR A REDE
class DatasetImagens(Dataset):
  def __init__(self, dados):
    self.images = [i["image"] for i in dados]; 
    self.labels = [l["label"] for l in dados]; 

  def __len__(self):
    return len(self.labels);

  def __getitem__(self, idx):
    image = self.images[idx]; 
    label = self.labels[idx]; 

    # Convert the image to a PyTorch tensor
    image = torch.from_numpy(image).float()
    # Convert the label to a PyTorch tensor
    label = torch.tensor(label).int()

    return image, label

In [ ]:
novas_imagens_barbacena = []
novas_imagens_rioverde = []

for image in imagens_barbacena:
  novas_imagens_barbacena.append({"image": image, "label": 0})  

for image in imagens_rioverde:
  novas_imagens_rioverde.append({"image": image, "label": 1})



In [ ]:
#21 IMAGENS TOTAIS PARA BARBACENA E 42 PARA RIO VERDE
#7 DE TREINO PARA BARBACENA (1/3) E 14 PARA RIOVERDE (~1/3)
treino_dataset = sample(novas_imagens_barbacena + novas_imagens_rioverde, 21); 
test_dataset = sample(novas_imagens_barbacena + novas_imagens_rioverde, 42);

#ESTA DANDO CERTO!!!!!

#ALTERANDO PARA A CLASSE DATASETIMAGE
treino_dataset = DatasetImagens(treino_dataset);
test_dataset = DatasetImagens(test_dataset);



In [ ]:
#DATALOADER 
batch_size = 16 
train_dataloader = DataLoader(treino_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#DEFININDO A ARQUITETURA DA MINHA REDE
class MinhaRede(nn.Module):
  def __init__(self):
    super(MinhaRede, self).__init__()

    # Definir a arquitetura
    self.rede = nn.Sequential(
        nn.Linear(600, 64),
        nn.ReLU(),
        #nn.Linear(200, 600),
        #nn.ReLU(),
        #nn.Linear(600, 64),
        #nn.ReLU(),
        nn.Linear(64, 2)
    )

  def forward(self, image):
    # Fluxo de passagem do dado
    saida = self.rede(image)
    return saida

In [ ]:
#INSTANCIANDO-A 
tam_entrada = 1 #28+14
model = MinhaRede()

In [ ]:
#FUNÇÃO DE PERDA E OTIMIZADOR
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#TREINANDO A REDE NEURAL
def train(model, dataloader, lossfunc, optimizer):
  model.train(True)
  cumloss = 0.0
  for image, label in dataloader:    
    #print(f"Imagem é: {image} e label é {label}");

    pred = model(image)
    loss = lossfunc(pred, label)

    #zera os gradientes acumulados
    optimizer.zero_grad()
    #computa os gradientes
    loss.backward()
    #anda, de fato, na direção que reduz o erro local
    optimizer.step()

    # loss é um tensor; item pra obter o float
    cumloss += loss.item() 
  
  return cumloss / len(dataloader)

In [ ]:
#FUNCAO PARA TESTAR A REDE NEURAL 
def test(model, dataloader, lossfunc):
  model.eval()
  
  cumloss = 0.0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.unsqueeze(1).float().to(device)
      y = y.unsqueeze(1).float().to(device)

      pred = model(X)
      loss = lossfunc(pred, y)
      cumloss += loss.item() 
  
  return cumloss / len(dataloader)

In [ ]:
#TREINANDO A REDE UMA VEZ
train(model, train_dataloader, criterion, optimizer)

RuntimeError: ignored

In [ ]:
#TREINANDO VARIAS VEZES 

epochs = 200
for t in range(epochs):
  train_loss = train(model, train_dataloader, criterion, optimizer)
  if t % 1000 == 0:
    print(f"Epoch: {t}; Train Loss: {train_loss}")
    

test_loss = test(model, test_dataloader, criterion)
print(f"Test Loss: {test_loss}")